In [1]:
! pip3 install pandas pycoingecko matplotlib scipy seaborn
import matplotlib.pyplot as plt
from scipy.stats import norm
import pandas as pd
import datetime as dt
import time
import statistics as stats
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random as rd
from pycoingecko import CoinGeckoAPI
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.tsa.ar_model import AutoReg
from statsmodels.formula.api import ols
from statsmodels.formula.api import wls
from statsmodels.iolib.summary2 import summary_col
from statsmodels.tsa.filters.hp_filter import hpfilter
from pandas.plotting import lag_plot

In [2]:
data_dir = '../data/crypto/'
filename = 'dogecoin_usd_elon.csv'
tweet_date = "2018-07-18 01:58:00"
date_format = "%Y-%m-%d %H:%M:%S"
tweet_datetime = pd.to_datetime(tweet_date,format=date_format)

In [3]:
df = pd.read_csv(data_dir+filename)

FileNotFoundError: [Errno 2] No such file or directory: '../data/crypto/dogecoin_usd_elon.csv'

In [ ]:
df

In [ ]:
def is_after_tweet(date,tweet_dt):
    if date < tweet_dt:
        return 0
    elif date > tweet_dt: 
        return 1
    else:
        pass
def days_since(date,tweet_dt):
    diff = date - tweet_dt
    return  diff.days + (diff.seconds / (3600 * 24))
def min_max(price,min_price,max_price):
    return (price - min_price)/(max_price-min_price)
def kernel(R,c,h):
    indicator = (np.abs(R-c) <= h).astype(float)
    return indicator * (1 - np.abs(R-c)/h)


In [ ]:
def gen_extra_cols(original_df,twt_dt,c=0,h=1):
    expanded_df = original_df.copy()
    expanded_df['before_after'] = expanded_df['date'].map(lambda x: is_after_tweet(x,twt_dt))
    expanded_df['days_since_tweet'] = expanded_df['date'].map(lambda x: days_since(x,twt_dt))
    expanded_df['min_max_prices'] = min_max(expanded_df['prices'],min(expanded_df['prices']),max(expanded_df['prices']))
    expanded_df['kernel_weighting'] = expanded_df['days_since_tweet'].map(lambda x: kernel(x,c,h)) 
    expanded_df['hp_prices'] = hp
    return expanded_df

In [ ]:
df = df.astype({'date': 'datetime64[ns]'})

In [ ]:
df = gen_extra_cols(df,tweet_datetime)

In [ ]:
def get_sub_df(original_df,twt_dt,window_size):
    sub_df = original_df.copy()
    sub_df['days_since_tweet'] = sub_df['date'].map(lambda x: days_since(x,twt_dt))
    sub_df = sub_df[(sub_df['days_since_tweet'] >= -window_size) & (sub_df['days_since_tweet'] <= window_size)].copy().reset_index(drop=True)
    return gen_extra_cols(sub_df,twt_dt)
def random_tweet_dt(original_df,window_size):
    sample_from = original_df['date'][0] + dt.timedelta(days=window_size)
    sample_to = original_df['date'][len(original_df.index)-1] - dt.timedelta(days=window_size)
    sampling_df = original_df[(original_df['date'] >= sample_from) & (original_df['date'] <= sample_to)]
    random_tweet = sampling_df.sample()
    random_tweet_date = original_df['date'][random_tweet.index.values.tolist()[0]]
    return random_tweet_date

In [ ]:
sub=get_sub_df(df,random_tweet_dt(df,1),1)

In [ ]:
def multi_window_models(original_df,twt_dt,window_sizes=range(1,8),with_kernel=False):
    models = []
    names = []
    for w in window_sizes:
        window_df = get_sub_df(original_df,twt_dt,w)
        if with_kernel:
            model = wls('min_max_prices ~ days_since_tweet * before_after',
                        window_df,
                        weights=kernel(window_df['days_since_tweet'],c=0,h=w)).fit()
        else:
            model = ols('min_max_prices ~ days_since_tweet * before_after',window_df).fit()
        models.append(model)
        names.append('± ' + str(w) + ' Days')
    return models, names
def multi_regression_tbl(models,names):
    return summary_col(
        models,
        stars=True,
        float_format='%0.3f',
        model_names=names,
        info_dict = {"N":lambda x: "{0:d}".format(int(x.nobs))})

In [ ]:
models,names = multi_window_models(df,tweet_datetime)
results_table = multi_regression_tbl(models,names)
results_table

In [ ]:
k_models,k_names = multi_window_models(df,tweet_datetime,with_kernel=True)
k_results_table = multi_regression_tbl(k_models,k_names)
k_results_table

In [ ]:
def perform_placebos(original_df,window_size,n=100,with_kernel=False):
    models = []
    for i in range(n):
        rand_twt_dt = random_tweet_dt(original_df,window_size)
        rand_window_df = get_sub_df(original_df,rand_twt_dt,window_size)
        if with_kernel:
            model = wls('min_max_prices ~ days_since_tweet * before_after',
                        rand_window_df,
                        weights=kernel(rand_window_df['days_since_tweet'],c=0,h=window_size)).fit()
        else:
            model = ols('min_max_prices ~ days_since_tweet * before_after',rand_window_df).fit()   
        models.append(model)
    return models

In [ ]:
rand_twt_dt = random_tweet_dt(df,1)
rand_window_df = get_sub_df(df,rand_twt_dt,1)
m = ols('min_max_prices ~ days_since_tweet * before_after',rand_window_df).fit()   
m.summary()

In [ ]:
placebos = perform_placebos(df,1,with_kernel=True,n=1000)

In [ ]:
placebo_coeffs = pd.Series(placebos).map(lambda x: x.params['before_after']).to_list()

In [ ]:
plt.hist(placebo_coeffs,bins=35,edgecolor='white',density=False)

In [ ]:
seven_days = get_sub_df(df,tweet_datetime,7)
seven_day_model = ols('min_max_prices ~ days_since_tweet * before_after',seven_days).fit()
ax = seven_days.plot(y='min_max_prices',x='days_since_tweet')
seven_days.assign(predictions=seven_day_model.fittedvalues).plot(x='days_since_tweet',y='predictions',ax=ax,color='C1')
plt.axvline(0,color='green',linestyle='--')

In [ ]:
def before_after_box_plot(original_df):
    axes=original_df.groupby(by='before_after').boxplot(column='prices',subplots=True)
    axes[0].set_title('')
    axes[1].set_title('')
    plt.suptitle('Comparative Box Plot of Coin Prices Before and After the Tweet')
    axes[0].set_xticklabels(['Coin Prices Before Tweet'])
    axes[1].set_xticklabels(['Coin Prices After Tweet'])
    return axes
before_after_box_plot(seven_days)

In [ ]:
one_day = get_sub_df(df,tweet_datetime,1)
one_day_model = ols('min_max_prices ~ days_since_tweet * before_after',one_day).fit()
ax = one_day.plot(y='min_max_prices',x='days_since_tweet')
one_day.assign(predictions=one_day_model.fittedvalues).plot(x='days_since_tweet',y='predictions',ax=ax,color='C1')
plt.axvline(0,color='green',linestyle='--')

In [ ]:
before_after_box_plot(one_day)